In [2]:
import pandas as pd
import numpy as np
from openai import OpenAI
import pandas as pd
import numpy as np
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4")
from tqdm import tqdm
import pickle
from transformers import AutoModel, AutoTokenizer
from torch.nn.functional import cosine_similarity
import torch.nn.functional as F
import torch
import csv

model = 'llama3.1:70b'
num_ctx = 15000


checkpoint="Salesforce/codet5p-110m-embedding"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True).to("cuda")
def get_code_embedding(code_snippet):
    """
    Generates embeddings for a given code snippet using a pre-trained model.

    Parameters:
    - code_snippet (str): The code for which embeddings are to be generated.
    - checkpoint (str): The model checkpoint to be used for embedding. Default is Salesforce/codet5p-110m-embedding.
    - device (str): Device to run the model on, either 'cuda' for GPU or 'cpu' for CPU. Default is 'cuda'.

    Returns:
    - torch.Tensor: Embedding tensor for the input code.
    """
    inputs = tokenizer.encode(code_snippet, return_tensors="pt").to("cuda")
    with torch.no_grad():
        embedding = model(inputs)[0]
    
    return embedding.cpu()

In [3]:
pickle_file_path = 'AMS_Df_promptsaturday.pkl'
# read the serialized DataFrame
with open(pickle_file_path, 'rb') as file:
    AMS_Df = pickle.load(file)
AMS_Df.head()
AMS_Df['EP'] = AMS_Df['EP'].apply(lambda i: i.split("_")[0])
def process_json_answer(json_answer):
    global counter
    all = []
    try:
        arrays = json_answer['Sinks']
        for i, array in enumerate(arrays, 1):
            joined = '\n'.join(array)
            all.append(joined)
    except:    
        return []
    counter += 1
    return all

# get first row of the DataFrame
# test = AMS_Df.iloc[0]["json_answer"]
# test

# Extract each array
# arrays = test['Sinks']

# for i, array in enumerate(arrays, 1):
#     joined = '\n'.join(array)
#     print(f"Array {i} (Joined):\n{joined}\n")
counter = 0
# Step 2: Apply to create a new column for joined json_answer
AMS_Df['sink_code'] = AMS_Df['json_answer'].apply(process_json_answer)

# display the first row first column of the DataFrame
print(f"columns with valid json  = {counter}")
AMS_Df.head()
AMS_Df.iloc[0]["sink_code"]


columns with valid json  = 48


['ApplicationInfo var210 = var207.getApplicationInfo(param5,1024,var19);\nServiceInfo var229 = new ServiceInfo();\nApplicationInfo var231 = new ApplicationInfo();\nvar229.applicationInfo = var231;\nString var235 = var210.packageName;\nvar234.packageName = var235;\nint var237 = var210.uid;\nvar236.uid = var237;',
 'ComponentName var238 = new ComponentName();\nString var239 = var210.packageName;\nString var241 = var138.getClassName();\nIntent var265 = setComponent(var238);',
 'ActivityManagerService var276 = this.mAm;\nboolean var282 = var276.isSingleton(var277,var278,var279,var280);',
 'IntentFirewall var421 = var420.mIntentFirewall;\nComponentName var422 = var408.name;\nApplicationInfo var423 = var408.appInfo;\nboolean var425 = var421.checkService(var422,param2,param7,param6,param4,var423);',
 'AppOpsManager var438 = this.mAm;\nAppOpsService var439 = var438.mAppOpsService;']

In [4]:
AMS_Df["embeddings"] = None
for index, row in tqdm(AMS_Df.iterrows(), total=len(AMS_Df), desc="Processing Rows"):
    code_snippets = row["sink_code"]
    embeddings = []
    for each in code_snippets:
        code_embedding = get_code_embedding(each)
        embeddings.append(code_embedding)
    AMS_Df.at[index, "embeddings"] = embeddings

# save with pickle not needed since its v.fast
# AMS_Df.columns
# store using pickle 
# with open('AMS_Df_embeddings.pkl', 'wb') as file:
#     pickle.dump(AMS_Df, file)


Processing Rows:   0%|          | 0/192 [00:00<?, ?it/s]

Processing Rows: 100%|██████████| 192/192 [00:02<00:00, 80.62it/s]


Compute Cosine Similarity # F.cosine_similarity(embedding2, embedding3, dim=0)

Get the most similar code(prolly ignore)

In [58]:

similarities = {}


for index1, row1 in tqdm(AMS_Df.iterrows(), total=len(AMS_Df), desc="Computing Similarities"):
    ep1_id = row1["EP"]  # name of the EP1
    ep1_embeddings = row1["embeddings"] # list of embeddings for the EP1
    ep1_sink_code = row1["sink_code"] # list of code snippets for the EP1
    
    closest_ep = None
    max_similarity = -1
    similar_sink_pair = None
    
    for index2, row2 in AMS_Df.iterrows():
        if index1 == index2:
            continue  
        
        ep2_id = row2["EP"] # name of the EP2
        ep2_embeddings = row2["embeddings"] # list of embeddings for the EP2
        ep2_sink_code = row2["sink_code"]  # list of code snippets for the EP2
        
        # Compute pairwise similarities between all embeddings of EP1 and EP2
        for i, emb1 in enumerate(ep1_embeddings):
            for j, emb2 in enumerate(ep2_embeddings):
                sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
                if sim > max_similarity:
                    max_similarity = sim
                    closest_ep = ep2_id
                    similar_sink_pair = (ep1_sink_code[i], ep2_sink_code[j])  
    
    similarities[ep1_id] = {
        "closest_ep": closest_ep,
        "max_similarity": max_similarity,
        "similar_sink_pair": similar_sink_pair
    }
    


Computing Similarities:   0%|          | 0/192 [00:00<?, ?it/s]/tmp/ipykernel_2409107/3436371449.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
Computing Similarities: 100%|██████████| 192/192 [00:17<00:00, 11.25it/s]


In [ ]:
for ep, details in similarities.items():
    if details['similar_sink_pair'] is None or details['closest_ep'] is None:
        print(f"EP {ep} has no similar EPs.")
    else:
        print(f"EP {ep} is most similar to EP {details['closest_ep']} with similarity {details['max_similarity']:.4f}")
        print(f"Similar sink codes:\nEP {ep}: \n{details['similar_sink_pair'][0]}\nEP \n{details['closest_ep']}: \n{details['similar_sink_pair'][1]}")
    print("\n\n")

Get >0.8 similar

In [59]:
similarities = {}

for index1, row1 in tqdm(AMS_Df.iterrows(), total=len(AMS_Df), desc="Computing Similarities"):
    ep1_id = row1["EP"] # name of the EP1
    ep1_embeddings = row1["embeddings"] # list of embeddings for the EP1
    ep1_sink_code = row1["sink_code"] # list of code snippets for the EP1
    
    similar_sink_pairs = []  # List to store all similar sink code pairs with similarity > 0.8
    
    for index2, row2 in AMS_Df.iterrows():
        if index1 == index2:
            continue  
        
        ep2_id = row2["EP"] # name of the EP2
        ep2_embeddings = row2["embeddings"] # list of embeddings for the EP2
        ep2_sink_code = row2["sink_code"] # list of code snippets for the EP2  
        
        # Compute similarities
        for i, emb1 in enumerate(ep1_embeddings):
            for j, emb2 in enumerate(ep2_embeddings):
                sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
                if sim > 0.8:  # Only consider pairs with similarity > 0.8
                    similar_sink_pairs.append({
                        "similarity": sim,
                        "ep1_code": ep1_sink_code[i],
                        "ep2_id": ep2_id,
                        "ep2_code": ep2_sink_code[j]
                    })
    
    # Store all similar pairs for the current EP
    similarities[ep1_id] = similar_sink_pairs

Computing Similarities:   0%|          | 0/192 [00:00<?, ?it/s]/tmp/ipykernel_2409107/4068078957.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
Computing Similarities: 100%|██████████| 192/192 [00:17<00:00, 11.27it/s]


In [ ]:
# Display results
for ep, similar_pairs in similarities.items():
    if not similar_pairs:
        print(f"EP {ep} has no similar EPs with similarity > 0.8.")
    else:
        print(f"EP {ep} has the following similar sink code pairs with similarity > 0.8:")
        for pair in similar_pairs:
            print(f"  - Similarity: {pair['similarity']:.4f}")
            print(f"    EP {ep}: \n{pair['ep1_code']}")
            print(f"    EP {pair['ep2_id']}: \n{pair['ep2_code']}")
            print()
    print("\n\n")

Write >0.8 similar and unique with top similarity

In [60]:
CSV_FILE = 'similarities1'

# Write the data to a CSV file
with open(CSV_FILE+ "_all" + ".csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write header
    writer.writerow(["EP1_ID", "EP2_ID", "EP1_Code", "EP2_Code", "Similarity"])

    # Iterate through the data and write each entry
    for ep, similar_pairs in similarities.items():
        if similar_pairs:
            for pair in similar_pairs:
                writer.writerow([
                    ep,
                    pair['ep2_id'],
                    pair['ep1_code'].replace("\n", " "),  # Replace newlines for better CSV readability
                    pair['ep2_code'].replace("\n", " "),  # Replace newlines for better CSV readability
                    pair['similarity']
                ])
        else:
            writer.writerow([ep, "No similar EPs with similarity > 0.8", "", "", ""])

# Writing the data to a CSV file
with open(CSV_FILE + "_uniq" + ".csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["EP", "EP2_ID", "Max_Similarity"])
    
    for ep, similar_pairs in similarities.items():
        if not similar_pairs:
            writer.writerow([ep, "No similar EPs", ""])
        else:
            max_similarity_per_ep2 = {}
            for pair in similar_pairs:
                ep2_id = pair['ep2_id']
                similarity = pair['similarity']
                if ep2_id not in max_similarity_per_ep2:
                    max_similarity_per_ep2[ep2_id] = similarity
                else:
                    max_similarity_per_ep2[ep2_id] = max(max_similarity_per_ep2[ep2_id], similarity)
            
            # Sorting the EP2 IDs by similarity in descending order
            sorted_ep2_ids = sorted(max_similarity_per_ep2.items(), key=lambda x: x[1], reverse=True)
            
            # Writing sorted EP2 IDs and their max similarity
            for ep2_id, max_similarity in sorted_ep2_ids:
                writer.writerow([ep, ep2_id, f"{max_similarity:.4f}"])

print(f"Data has been written to {CSV_FILE}")


# print("Data has been written to 'similarities.csv'.")

Data has been written to similarities1


In [ ]:
for ep, similar_pairs in similarities.items():
    if not similar_pairs:
        print(f"EP {ep} has no similar EPs with similarity > 0.8.")
    else:
        print(f"EP {ep} has the following similar sink code pairs with similarity > 0.8:")
        for pair in similar_pairs:
            print(f"  - Similarity: {pair['similarity']:.4f}")
            print(f"    EP {ep}")
            print(f"    EP {pair['ep2_id']}")
            print()
    print("\n\n")

1 EP with maximum similarity value

In [ ]:

for ep, similar_pairs in similarities.items():
    if not similar_pairs:
        print(f"EP {ep} has no similar EPs with similarity > 0.8.")
    else:
        print(f"EP {ep} has the following similar sink code pairs with similarity > 0.8:")
        
        max_similarity_per_ep2 = {}
        for pair in similar_pairs:
            ep2_id = pair['ep2_id']
            similarity = pair['similarity']
            if ep2_id not in max_similarity_per_ep2:
                max_similarity_per_ep2[ep2_id] = similarity
            else:
                max_similarity_per_ep2[ep2_id] = max(max_similarity_per_ep2[ep2_id], similarity)
        print("Unique EP2 IDs and their maximum similarity:")
        sorted_ep2_ids = sorted(max_similarity_per_ep2.items(), key=lambda x: x[1], reverse=True)
        for ep2_id, max_similarity in sorted_ep2_ids:
            print(f"EP2 ID: {ep2_id}, Max Similarity: {max_similarity:.4f}")
    print("\n\n")



In [ ]:
print(AMS_Df[AMS_Df["EP"] == "updateConfiguration_ActivityManagerService_2"]["sink_code"].tolist())

Write APIs

In [ ]:
unformated_APIs = AMS_Df["EP"].unique()
formated_APIs = [i.split("_")[0] for i in unformated_APIs]
# write formated apis to a csv
pd.DataFrame(formated_APIs).to_csv("formated_APIs.csv", index=False)